In [ ]:
cp drive/MyDrive/digital_breakthrough_altai_time.zip ./

In [ ]:
!unzip digital_breakthrough_altai_time.zip
!unzip digital_breakthrough_altai_time/AltGU.zip
!unzip digital_breakthrough_altai_time/test_dataset_test.zip
!unzip digital_breakthrough_altai_time/train_dataset_train.zip

Archive:  digital_breakthrough_altai_time.zip
 extracting: digital_breakthrough_altai_time/AltGU.zip  
 extracting: digital_breakthrough_altai_time/test_dataset_test.zip  
 extracting: digital_breakthrough_altai_time/train_dataset_train.zip  
Archive:  digital_breakthrough_altai_time/AltGU.zip
  inflating: employees.csv           
  inflating: sample_solution.csv     
  inflating: АлтГУ. Всерос.pdf  
  inflating: АлтГУ_baseline.ipynb  
Archive:  digital_breakthrough_altai_time/test_dataset_test.zip
  inflating: test_comments.csv       
  inflating: test_issues.csv         
Archive:  digital_breakthrough_altai_time/train_dataset_train.zip
  inflating: train_issues.csv        
  inflating: train_comments.csv      


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 13 kB/s 


In [ ]:
import catboost
from catboost import Pool, cv
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
train_comments = pd.read_csv("train_comments.csv")
train_issues = pd.read_csv("train_issues.csv")
employees = pd.read_csv("employees.csv")

test_comments = pd.read_csv("test_comments.csv")
test_issues = pd.read_csv("test_issues.csv")

In [ ]:
# train_comments.head()

In [ ]:
# train_issues.head()

In [ ]:
# employees.head()

In [ ]:
# test_comments.head()

In [ ]:
# test_issues.head()

In [ ]:
# for column in ("assignee_id", "project_id", "creator_id"):
#     diff_values = set(test_issues[column].unique()).difference(set(train_issues[column].unique()))
#     print(column)
#     print(test_issues[test_issues[column].apply(lambda x: x in diff_values)])
#     print("-" * 80, "\n\n")

In [ ]:
# TODO добавить праздник ли
# убрать id
# попробовать просто без кросс-валидации, а трейн и валидацию

In [ ]:
def process_df(df, comment_df=None):
    df["created"] = df["created"].astype("datetime64")
    df["year"] = df["created"].apply(lambda x: x.year)
    df["month"] = df["created"].apply(lambda x: x.month)
    df["day"] = df["created"].apply(lambda x: x.day)
    df["hour"] = df["created"].apply(lambda x: x.hour)
    df["dayofweek"] = df["created"].apply(lambda x: x.dayofweek)
    df["minute"] = df["created"].apply(lambda x: x.minute)
    # df["created"] = pd.to_datetime(df["created"])
    df["key_first_part"] = df["key"].str.split("-").str[0]
    df["key_second_part"] = df["key"].str.split("-").str[1].astype(int)
    if comment_df is not None:
        comment_count_dict = comment_df["issue_id"].value_counts().to_dict()
        df["comment_count"] = df["id"].apply(
            lambda x: 0 if x not in comment_count_dict else comment_count_dict[x])
    df = df.merge(
        employees, how="left", left_on="assignee_id", right_on="id", suffixes=["", "_assignee"])
    df = df.merge(
        employees, how="left", left_on="creator_id", right_on="id", suffixes=["", "_creator"])
    df = df.fillna(-1)
    for col in("position", "position_creator"):
        df[col] = df[col].replace(-1, "")
    drop_cols = ["id", "key", "created"]
    drop_cols += ["id_assignee", "full_name", "id_creator", "full_name_creator"]
    df = df.drop(drop_cols, axis=1)
    return df

def predict(dataset, labels=None):
    preds = []
    for model in models:
        pred = model.predict(dataset)
        preds.append(pred)
    mean_preds = np.array(preds).mean(axis=0)
    if labels is not None:
        score = r2_score(labels, mean_preds)
        print(score)
    return mean_preds

In [ ]:
train_comments = pd.read_csv("train_comments.csv")
train_issues = pd.read_csv("train_issues.csv")
employees = pd.read_csv("employees.csv")

test_comments = pd.read_csv("test_comments.csv")
test_issues = pd.read_csv("test_issues.csv")

train_issues = process_df(train_issues, train_comments)
test_issues = process_df(test_issues, test_comments)

train, test = train_test_split(train_issues, test_size=0.1, random_state=42)

text_features = ["summary"]
text_features += ["position", "position_creator"]
cat_features = ["key_first_part"]
cat_features += ["english_level", "english_level_creator",
                "hiring_type", "hiring_type_creator", "payment_type", "payment_type_creator",
                "salary_calculation_type", "salary_calculation_type_creator"]

whole_dataset = Pool(
    data=train_issues[[c for c in train_issues.columns if c != "overall_worklogs"]],
    label=train_issues["overall_worklogs"],
    text_features=text_features,
    cat_features=cat_features
)

train_dataset = Pool(
    data=train[[c for c in train.columns if c != "overall_worklogs"]],
    label=train["overall_worklogs"],
    text_features=text_features,
    cat_features=cat_features
)

test_dataset = Pool(
    data=test[[c for c in test.columns if c != "overall_worklogs"]],
    label=test["overall_worklogs"],
    text_features=text_features,
    cat_features=cat_features
)

In [ ]:
params = {"iterations": 100,
          "depth": 2,
          "loss_function": "RMSE",
          "verbose": False,
          }

scores, models = cv(
    train_dataset,
    params,
    fold_count=3,
    return_models=True)

preds = predict(test_dataset, test["overall_worklogs"])

Training on fold [0/3]

bestTest = 53530.0879
bestIteration = 99

Training on fold [1/3]

bestTest = 51518.98322
bestIteration = 42

Training on fold [2/3]

bestTest = 133818.8445
bestIteration = 99

0.00044275630757173445


In [ ]:
params = {"iterations": 1000,
          "depth": 2,
          "loss_function": "RMSE",
          "verbose": False,
          "early_stopping_rounds": 20,
          }

scores, models = cv(
    train_dataset,
    params,
    fold_count=3,
    return_models=True)

preds = predict(test_dataset, test["overall_worklogs"])

Training on fold [0/3]

bestTest = 53483.20152
bestIteration = 103

Training on fold [1/3]

bestTest = 51518.98322
bestIteration = 42

Training on fold [2/3]

bestTest = 127988.6989
bestIteration = 342

0.006226069265458656


In [ ]:
params = {"iterations": 2000,
          "depth": 8,
          "loss_function": "RMSE",
          "verbose": False,
          "early_stopping_rounds": 20,
          }

scores, models = cv(
    train_dataset,
    params,
    fold_count=3,
    return_models=True)

preds = predict(test_dataset, test["overall_worklogs"])

Training on fold [0/3]

bestTest = 49678.06325
bestIteration = 311

Training on fold [1/3]

bestTest = 45881.68894
bestIteration = 96

Training on fold [2/3]


In [ ]:
params = {"iterations": 2000,
          "depth": 8,
          "loss_function": "RMSE",
          "verbose": False,
          "early_stopping_rounds": 20,
          }

scores, models = cv(
    whole_dataset,
    params,
    fold_count=3,
    return_models=True)

preds = predict(test_dataset, test["overall_worklogs"])

Training on fold [0/3]

bestTest = 78421.25346
bestIteration = 491

Training on fold [1/3]

bestTest = 132171.5992
bestIteration = 21

Training on fold [2/3]

bestTest = 53581.47481
bestIteration = 72

0.2899170008098896


In [ ]:
test_dataset = Pool(
    data=test_issues,
    text_features=["summary"],
    cat_features=["key_first_part"]
)

CatBoostError: ignored

In [ ]:
preds = []
for model in models:
    pred = model.predict(test_issues)
    preds.append(pred)

In [ ]:
preds[0].shape

In [ ]:
mean_preds = np.array(preds).mean(axis=0)

In [ ]:
solution = pd.read_csv("sample_solution.csv")
solution

In [ ]:
solution["overall_worklogs"] = mean_preds

In [ ]:
solution

,id,overall_worklogs
0,675975,15688.120388
1,675972,13080.313544
2,675965,13654.941526
3,675961,11840.731011
4,675955,15208.335575
...,...,...
1065,702545,13326.166968
1066,702528,14724.047565
1067,702499,15344.223859
1068,702376,15831.302260


In [ ]:
solution.to_csv("solution.csv", index=None)